In [130]:
import numpy as np
import spacy
import torch
from torch import nn
from datasets import load_dataset, Dataset
import os

In [42]:
def tokenize(messages):
    tokens = set()
    message_to_tokens = []
    for message in messages:
        doc = nlp(message)
        message_tokens = {token.text.lower() for token in doc if
                          token.pos_ in {'ADJ', 'ADV', 'INTJ', 'NOUN', 'PROPN', 'VERB'}}
        tokens.update(message_tokens)
        message_to_tokens.append(message_tokens)
    return list(tokens), message_to_tokens

In [89]:
def encode_x(vocabulary, tokens):
    vector = [0] * len(vocabulary)
    for token in tokens:
        index = vocabulary[token]
        vector[index] = 1
    return vector


def encode_y(label):
    vector = [0] * 3
    vector[label] = 1
    return vector

In [45]:
nlp = spacy.load('en_core_web_sm')
dataset = load_dataset('Sp1786/multiclass-sentiment-analysis-dataset')
train_dataset: Dataset = dataset['train']

tokens, message_to_tokens = tokenize(train_dataset['text'])

vocabulary = {token: index for index, token in enumerate(tokens)}
print(len(vocabulary))

29050


In [90]:
xs = [encode_x(vocabulary, tokens) for tokens in message_to_tokens]
ys = [encode_y(label) for label in train_dataset['label']]

In [110]:
def create_batch(xs, ys, batch_size):
    random_indices = np.random.choice(len(xs), batch_size)
    return torch.tensor(np.stack([xs[index] for index in random_indices]), dtype=torch.float32), torch.tensor(
        np.stack([ys[index] for index in random_indices]), dtype=torch.float32)

In [125]:
def determine_device():
    if torch.cuda.is_available():
        return 'cuda'
    elif torch.backends.mps.is_available():
        return 'mps'
    else:
        return 'cpu'

device = determine_device()
print(f'Device is {device}')

Device is mps


In [136]:
model = nn.Sequential(
    nn.Linear(len(vocabulary), 256),
    nn.Linear(256, 3),
    nn.Softmax(-1)
).to(device)

batch_size = 512
loss_fn = nn.CrossEntropyLoss(reduction="mean")
optimizer = torch.optim.Adam(model.parameters(), lr=5e-3)

cwd = os.getcwd()
print(cwd)

for epoch in range(2000):
    x_batch, y_batch = create_batch(xs, ys, batch_size)
    prediction = model(x_batch.to(device))
    loss = loss_fn(prediction, y_batch.to(device))

    loss.backward()
    optimizer.step()
    model.zero_grad()

    if epoch % 10 == 0:
        print(epoch, loss.item())
    if epoch % 100 == 0:
        model_file_name = os.path.join(cwd, "models", "model_" + str(epoch) + ".pt")
        torch.save(model.state_dict(), model_file_name)
        print("Model has been saved as", model_file_name)


/Users/yaskovdev/dev/git_home/ai-sandbox/sentiment-analysis
0 1.0978257656097412
Model has been saved as /Users/yaskovdev/dev/git_home/ai-sandbox/sentiment-analysis/models/model_0.pt
10 0.9888988733291626
20 0.8937515020370483
30 0.8481156229972839
40 0.8082658648490906
50 0.8128935098648071
60 0.7954080104827881
70 0.7640846967697144
80 0.7446404695510864
90 0.7770837545394897
100 0.7855230569839478
Model has been saved as /Users/yaskovdev/dev/git_home/ai-sandbox/sentiment-analysis/models/model_100.pt
110 0.7482659220695496
120 0.7385355234146118
130 0.7428759932518005
140 0.7526922225952148
150 0.7171924114227295
160 0.7254508137702942
170 0.6995365619659424
180 0.7340483069419861
190 0.7070382833480835
200 0.6975364685058594
Model has been saved as /Users/yaskovdev/dev/git_home/ai-sandbox/sentiment-analysis/models/model_200.pt
210 0.6975405812263489
220 0.725130558013916
230 0.6932360529899597
240 0.7086825966835022
250 0.70079505443573
260 0.7058854103088379
270 0.7169591188430786


KeyboardInterrupt: 

In [139]:
model.load_state_dict(torch.load(os.path.join(cwd, "models", 'model_1300.pt'), map_location=torch.device(device)))
model.eval()

In [192]:
message = "My computer is totally broken"
tokens, message_to_tokens = tokenize([message])
x = encode_x(vocabulary, tokens)
y = model(torch.tensor(x, dtype=torch.float32).to(device))
answer = torch.multinomial(y, 1, replacement=True)
print(['negative', 'neutral', 'positive'][answer])

negative


In [185]:
torch.onnx.export(model, torch.tensor(x, dtype=torch.float32).to(device), "model.onnx")